<a href="https://colab.research.google.com/github/JPalacios-8/DL-Proyecto-1/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Paso 1: Instalar medmnist
!pip install medmnist tensorflow

# Paso 2: Importar librerías
import medmnist
from medmnist import INFO
from medmnist import ChestMNIST
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

data_flag = 'chestmnist'
info = INFO[data_flag]
download = True


#PUNTO 2 -> DISTRIBUCION PREDETERMINADA DEL DATASET
train_dataset = ChestMNIST(split='train', download=download)
val_dataset = ChestMNIST(split='val', download=download)
test_dataset = ChestMNIST(split='test', download=download)

# Convertir a arrays NumPy manualmente
x_train, y_train = train_dataset.imgs, train_dataset.labels
x_val, y_val = val_dataset.imgs, val_dataset.labels
x_test, y_test = test_dataset.imgs, test_dataset.labels


# Normalizar: de [0, 255] → [0, 1]
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

# Expandir dimensión si es necesario (canal = 1)
x_train = np.expand_dims(x_train, -1)
x_val = np.expand_dims(x_val, -1)
x_test = np.expand_dims(x_test, -1)

print("Train shape:", x_train.shape)
print("Validation shape:", x_val.shape)
print("Test shape:", x_test.shape)


Train shape: (78468, 28, 28, 1)
Validation shape: (11219, 28, 28, 1)
Test shape: (22433, 28, 28, 1)


In [24]:
#PUNTO 1
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(14, activation='sigmoid')  # 3 etiquetas multilabel
])



In [26]:
# Paso 5: Compilar el modelo
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # porque es multilabel
    metrics=['accuracy', tf.keras.metrics.AUC()]
)



In [27]:
test_loss, test_acc, test_auc = model.evaluate(x_test, y_test)
print(f"\nTest Accuracy: {test_acc:.4f} - AUC: {test_auc:.4f}")


702/702 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0250 - auc_2: 0.4230 - loss: 0.8123

Test Accuracy: 0.0240 - AUC: 0.4250


In [28]:
#PUNTO 3: PRUEBAS AUTOMATICAS CON KERAS TUNER
!pip install keras-tuner
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.Input(shape=(28, 28, 1)))
    model.add(layers.Flatten())

    # Número de capas ocultas
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Choice('units_' + str(i), [64, 128, 256]),
            activation=hp.Choice('activation_' + str(i), ['relu', 'tanh', 'sigmoid'])
        ))

    model.add(layers.Dense(14, activation='sigmoid'))  # Salida multi-label

    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd']),
        loss='binary_crossentropy',
        metrics=['accuracy']#NOTA 1
    ) #

    return model


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.4 MB/s eta 0:00:00


In [29]:
#Random search
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # puedes subirlo si tienes tiempo
    executions_per_trial=1,
    directory='tuner_results',
    project_name='mlp_chestmnist'
)


In [30]:
# Entrenamiento con búsqueda
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val), batch_size=64)


Trial 5 Complete [00h 01m 36s]
val_accuracy: 0.5516534447669983

Best val_accuracy So Far: 0.5516534447669983
Total elapsed time: 00h 07m 03s


In [31]:
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluar en test
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"Mejor modelo - Accuracy: {test_acc:.4f}")


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


702/702 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5431 - loss: 0.1878
Mejor modelo - Accuracy: 0.5427
